Libaries

In [7]:
!pip install python-dotenv gradio
import os
import dotenv
from dotenv import load_dotenv
from openai import OpenAI
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import gradio as gr
import json
import math

Setting up API KEY access.

In [2]:
# Load .env file
load_dotenv(dotenv_path='/content/drive/MyDrive/github_projects/stock_price_reasoning_model/.env')
API_KEY = os.getenv("OPENAI_API_KEY")

# Confirm that the API key has been loaded (print statement for testing)
print("API Key loaded:", bool(API_KEY))

API Key loaded: True


Checking available models from API.

In [3]:
client = OpenAI(api_key=API_KEY)

try:
    models = client.models.list()
    print("Available model names:")
    for model in models:
        print(model.id)
except Exception as e:
    print(f"Error fetching models: {str(e)}")

Available model names:
gpt-4o-mini-audio-preview
gpt-4o-mini-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview
dall-e-2
o1-mini-2024-09-12
o1-preview-2024-09-12
o1-mini
o1-preview
chatgpt-4o-latest
whisper-1
gpt-4-turbo
gpt-4-turbo-preview
gpt-4o-audio-preview
gpt-4o-audio-preview-2024-10-01
babbage-002
omni-moderation-latest
omni-moderation-2024-09-26
tts-1-hd-1106
gpt-4o-2024-05-13
tts-1-hd
dall-e-3
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-11-20
gpt-4o-2024-08-06
gpt-4-turbo-2024-04-09
gpt-4o
tts-1
gpt-3.5-turbo-16k
tts-1-1106
davinci-002
gpt-3.5-turbo-1106
gpt-4o-mini-realtime-preview-2024-12-17
gpt-3.5-turbo-instruct
gpt-4o-realtime-preview-2024-10-01
gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-0125
gpt-4o-audio-preview-2024-12-17
gpt-4o-realtime-preview-2024-12-17
gpt-3.5-turbo
text-embedding-3-large
gpt-4o-realtime-preview
text-embedding-3-small
gpt-4-0125-preview
gpt-4
text-embedding-ada-002
gpt-4-1106-preview
gpt-4-0613


Displaying stock price and dividend amount.

In [ ]:
# Function to plot dividend payouts with stock close price
def plot_dividends(ticker):
    try:
        # Fetch the ticker object
        ticker_obj = yf.Ticker(ticker)

        # Fetch dividend data
        dividends = ticker_obj.dividends
        if dividends.empty:
            return "No dividend data available for this stock."

        # Fetch historical stock prices (ensuring it covers dividend dates)
        stock_history = ticker_obj.history(start=dividends.index.min(), end=dividends.index.max())

        # Align stock close prices with dividend dates (forward-fill missing prices)
        close_prices = stock_history["Close"].reindex(dividends.index, method="ffill")

        # Create a Plotly figure
        fig = go.Figure()

        # Add dividend payouts
        fig.add_trace(go.Scatter(
            x=dividends.index,
            y=dividends,
            mode='lines+markers',
            name='Dividends ($)',
            marker=dict(size=8, color='red', symbol='circle'),
            hovertemplate="Date: %{x}<br>Dividend: $%{y:.2f}<br>Close Price: $%{customdata:.2f}",
            customdata=close_prices  # Ensures close price is shown on hover
        ))

        # Add stock close prices
        fig.add_trace(go.Scatter(
            x=close_prices.index,
            y=close_prices,
            mode='lines+markers',
            name='Close Price ($)',
            marker=dict(size=8, color='blue', symbol='square'),
            hovertemplate="Date: %{x}<br>Dividend: $%{y:.2f}<br>Close Price: $%{customdata:.2f}",
            customdata=dividends  # Ensures dividend is shown on hover
        ))

        # Update layout
        fig.update_layout(
            title=f'{ticker} Dividend Payouts & Close Prices Over Time',
            xaxis_title='Date',
            yaxis_title='Amount ($)',
            template='plotly_dark',
            width=1200,
            height=800,
            yaxis=dict(dtick=5)  # Set tick spacing to 5
        )

        return fig

    except Exception as e:
        return f"Error: {e}"

# Create a Gradio Interface
gr_interface = gr.Blocks()

with gr_interface:
    gr.Markdown("# 📈 Dividend & Stock Price Plotter")
    gr.Markdown("Enter a ticker symbol to see dividend payouts over time alongside stock closing prices.")

    with gr.Row():
        ticker_input = gr.Textbox(label="Enter Ticker Symbol")
        submit_btn = gr.Button("Generate Graph")

    plot_output = gr.Plot(label="Dividend Payouts & Close Prices")

    submit_btn.click(fn=plot_dividends, inputs=ticker_input, outputs=plot_output)

# Launch the Gradio interface
gr_interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d5785f196c26f44b59.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import yfinance as yf

# Define the ticker
ticker = "ENB.TO"

# Fetch the ticker object
ticker_obj = yf.Ticker(ticker)

# Fetch the dividend data (dates and amounts)
dividends = ticker_obj.dividends

# Print the dividends and their respective dates
print("Dividends Paid Out (Date and Amount):")
print(dividends)

# Print the data types of the columns
print("\nData Types of Columns:")
print(dividends.dtypes)

print(ticker_obj.info)

Dividends Paid Out (Date and Amount):
Date
1995-02-07 00:00:00-05:00    0.0625
1995-05-23 00:00:00-04:00    0.0625
1995-08-11 00:00:00-04:00    0.0625
1995-11-17 00:00:00-05:00    0.0625
1996-02-14 00:00:00-05:00    0.0625
                              ...  
2023-11-14 00:00:00-05:00    0.8880
2024-02-14 00:00:00-05:00    0.9150
2024-05-14 00:00:00-04:00    0.9150
2024-08-15 00:00:00-04:00    0.9150
2024-11-15 00:00:00-05:00    0.9150
Name: Dividends, Length: 120, dtype: float64

Data Types of Columns:
float64
{'address1': '200, Fifth Avenue Place', 'address2': '425 - 1st Street S.W.', 'city': 'Calgary', 'state': 'AB', 'zip': 'T2P 3L8', 'country': 'Canada', 'phone': '403-231-3900', 'fax': '403-231-3920', 'website': 'https://www.enbridge.com', 'industry': 'Oil & Gas Midstream', 'industryKey': 'oil-gas-midstream', 'industryDisp': 'Oil & Gas Midstream', 'sector': 'Energy', 'sectorKey': 'energy', 'sectorDisp': 'Energy', 'longBusinessSummary': 'Enbridge Inc., together with its subsidiaries,

In [8]:
import gradio as gr
import yfinance as yf
import json
import openai
import math
import plotly.graph_objects as go

# Initialize OpenAI API
openai.api_key = API_KEY  # Replace with your API key

# Function to fetch stock/ETF data from Yahoo Finance
def fetch_stock_data(ticker_symbol):
    stock = yf.Ticker(ticker_symbol)
    stock_info = stock.info  # Fetch all available data

    # Select relevant features for reasoning
    relevant_data = {
        "symbol": stock_info.get("symbol", ""),
        "sector": stock_info.get("sector", "N/A"),
        "previous_close": stock_info.get("previousClose", "N/A"),
        "dividend_yield": stock_info.get("dividendYield") or stock_info.get("yield", "N/A"),
        "fifty_day_avg": stock_info.get("fiftyDayAverage", "N/A"),
    }

    # Prepare values for dividend calculation
    previous_close = stock_info.get("previousClose", 0)
    dividend_yield = stock_info.get("dividendYield") or stock_info.get("yield", 0)
    fifty_day_avg = stock_info.get("fiftyDayAverage", 0)

    return relevant_data, previous_close, dividend_yield, fifty_day_avg

# Function to call OpenAI API for stock analysis
def analyze_stock_with_o1_mini(stock_data):
    user_message = {
        "role": "user",
        "content": (
            "You are an AI stock advisor. Based on the given stock data, provide an analysis, firtsly mention the company name and take into account their industry"
            "on whether it is a good investment. Consider valuation, volatility, dividend yield, "
            "and market trends. Give reasoning for each buy, hold, or sell recommendation"
            "Please provide the analysis using plain text, with no formatting (no bold, italics, or other styles)."
            f"Here is the stock/ETF data:\n{json.dumps(stock_data, indent=2)}"
        )
    }

    try:
        response = client.chat.completions.create(
            model="o1-mini",
            messages=[user_message]
        )

        return response.choices[0].message.content
    except Exception as e:
        return f"Error in AI response: {str(e)}"

# Function to generate stock analysis
def get_stock_analysis(ticker):
    stock_data, previous_close, dividend_yield, fifty_day_avg = fetch_stock_data(ticker)

    dividend_yield = dividend_yield * 100
    stock_data['dividend_yield'] = dividend_yield

    # Calculate annual dividend payout based on $1000 investment
    investment_amount = 1000
    average_price = fifty_day_avg if fifty_day_avg else previous_close
    shares_owned = math.floor(investment_amount / average_price)
    dividend_per_share = (dividend_yield / 100) * average_price
    annual_dividend = shares_owned * dividend_per_share

    formatted_data = {
        "Ticker": stock_data['symbol'],
        "Sector": stock_data['sector'],
        "Previous Close": f"${stock_data['previous_close']:.2f}",
        "Dividend Yield": f"{stock_data['dividend_yield']:.2f}%",
        "Annual Dividend Payout (for $1000 investment)": f"${annual_dividend:.2f}"
    }

    ai_analysis = analyze_stock_with_o1_mini(stock_data)

    return formatted_data, ai_analysis

# Function to plot dividend payouts with stock close price
def plot_dividends(ticker):
    try:
        ticker_obj = yf.Ticker(ticker)
        dividends = ticker_obj.dividends
        if dividends.empty:
            return "No dividend data available for this stock."

        stock_history = ticker_obj.history(start=dividends.index.min(), end=dividends.index.max())
        close_prices = stock_history["Close"].reindex(dividends.index, method="ffill")

        fig = go.Figure()

        # Add dividend payouts
        fig.add_trace(go.Scatter(
            x=dividends.index,
            y=dividends,
            mode='lines+markers',
            name='Dividends ($)',
            marker=dict(size=8, color='red', symbol='circle'),
            hovertemplate="Date: %{x}<br>Dividend: $%{y:.2f}<br>Close Price: $%{customdata:.2f}",
            customdata=close_prices
        ))

        # Add stock close prices
        fig.add_trace(go.Scatter(
            x=close_prices.index,
            y=close_prices,
            mode='lines+markers',
            name='Close Price ($)',
            marker=dict(size=8, color='blue', symbol='square'),
            hovertemplate="Date: %{x}<br>Close Price: $%{y:.2f}",
        ))

        fig.update_layout(
            title=f'{ticker} Dividend Payouts & Close Prices Over Time',
            xaxis_title='Date',
            yaxis_title='Amount ($)',
            template='plotly_dark',
            width=1200,
            height=700,
            yaxis=dict(dtick=5)
        )

        return fig
    except Exception as e:
        return f"Error: {e}"

# Gradio Interface with Tabs
gr_interface = gr.Blocks()

with gr_interface:
    gr.Markdown("<h1 style='text-align: center;'>Welcome to BHSL Strategies: Buy High 📈, Sell Low 📉</h1>")
    gr.Markdown("<h3 style='text-align: center;'>Who doesn't love market fluctuations!</h3>")
    gr.Markdown("Enter a stock/ETF ticker to get analysis, recommendations, and dividend insights.")

    with gr.Row():
        ticker_input = gr.Textbox(label="Enter Ticker Symbol")
        submit_btn = gr.Button("Generate Analysis & Graph")

    with gr.Tab("Analysis"):
        with gr.Row():
            stock_output = [gr.JSON(label="Stock Data"), gr.Textbox(label="Quant Recommendation")]

    with gr.Tab("Dividend Plot"):
        plot_output = gr.Plot(label="Dividend Payouts & Close Prices")

    submit_btn.click(fn=get_stock_analysis, inputs=ticker_input, outputs=stock_output)
    submit_btn.click(fn=plot_dividends, inputs=ticker_input, outputs=plot_output)

# Launch the Gradio interface
gr_interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ebeaa86376a913bd5e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
